In [2]:
from pyarrow import csv

In [3]:
read_options = csv.ReadOptions(
               column_names=["time","source user@domain","destination user@domain", \
                             "source computer","destination computer","authentication type", \
                             "logon type","authentication orientation","success/failure","day"])


In [4]:
table_file = csv.read_csv('../lm-vol/LANL_test_unique_v2.csv', read_options=read_options)

ArrowInvalid: CSV parse error: Expected 10 columns, got 3: source user@domain,destination computer,source computer

In [ ]:
help(table_file.filter)

In [ ]:
pandas_lanl = table_file.to_pandas()

In [ ]:
pandas_lanl.shape

In [ ]:
pandas_lanl.columns

In [8]:
import pandas as pd 
import torch
from torch import Tensor

In [9]:
pandas_lanl.sample(5)

,time,source user@domain,destination user@domain,source computer,destination computer,authentication type,logon type,authentication orientation,success/failure,day
348405497,3387188,U53@DOM1,U53@DOM1,C612,C612,?,Network,LogOff,Success,40
602164002,4433436,U6752@DOM1,U6752@DOM1,C10,C10,?,Network,LogOff,Success,52
634846394,4605643,C12757$@DOM1,C12757$@DOM1,C612,C612,?,Network,LogOff,Success,54
624831862,4547984,U346@DOM1,U346@DOM1,C1183,C92,Kerberos,Network,LogOn,Success,53
443144291,3762135,U1457@?,U1457@?,C552,C552,?,?,TGT,Fail,44


In [10]:
pandas_lanl = pandas_lanl.rename(columns={'source user@domain': "src_user", 'destination user@domain': "dest_user"})

In [11]:
df = pandas_lanl ##shallow copy lol

In [12]:
df.columns

Index(['time', 'src_user', 'dest_user', 'source computer',
       'destination computer', 'authentication type', 'logon type',
       'authentication orientation', 'success/failure', 'day'],
      dtype='object')

In [13]:
df = df[(df['src_user'].str.contains('LOCAL')==False) & (df['src_user'].str.contains('SYSTEM')==False) & 
        (df['src_user'].str.contains('ANONYMOUS')==False) & (df['src_user'].str.contains('ADMINISTRARTOR')==False) &
        (df['src_user'].str.contains('$')==False) 
       ]

df = df[(df['dest_user'].str.contains('LOCAL')==False) & (df['dest_user'].str.contains('SYSTEM')==False) & 
        (df['dest_user'].str.contains('ANONYMOUS')==False) & (df['dest_user'].str.contains('ADMINISTRARTOR')==False) &
        (df['dest_user'].str.contains('$')==False) 
       ]


In [14]:
df.shape

(0, 10)

In [16]:
pandas_lanl.sample(2)

,time,src_user,dest_user,source computer,destination computer,authentication type,logon type,authentication orientation,success/failure,day
620080501,4521322,U341@DOM1,U341@DOM1,C612,C612,?,Network,LogOff,Success,53
99001974,1490866,C2070$@DOM1,C2070$@DOM1,C2071,C2071,?,?,TGT,Success,18


In [1]:
####decide to play with unique test data 
import pandas as pd 
import torch
from torch import Tensor

In [2]:

pandas_lanl = pd.read_csv("../lm-vol/LANL_test_NTLM_Pikachu_filter.csv")

/tmp/ipykernel_450327/2683578649.py:1: DtypeWarning: Columns (0,9) have mixed types. Specify dtype option on import or set low_memory=False.
  pandas_lanl = pd.read_csv("../lm-vol/LANL_test_NTLM_Pikachu_filter.csv")


In [3]:
pandas_lanl.shape

(3590236, 12)

In [4]:
client_service = pandas_lanl[['source user@domain','destination computer']].copy()
client_service.sample(5)

,source user@domain,destination computer
2718838,U9336@DOM1,C92
2317722,U14@DOM1,C433
1369727,U7539@DOM1,C1015
1930436,U4003@DOM1,C5618
2664098,U7159@DOM1,C625


In [5]:
#df.rename(columns={"A": "a", "B": "c"})
client_service = client_service.rename(columns={"source user@domain":"client","destination computer":"service"})

In [6]:
unique_client_id = client_service['client'].unique()
unique_client_id = pd.DataFrame(data={
     'clientId': unique_client_id,
     'mappedID': pd.RangeIndex(len(unique_client_id)),
 })

print(unique_client_id)



unique_service_id = client_service['service'].unique()
unique_service_id = pd.DataFrame(data={
     'serviceId': unique_service_id,
     'mappedID': pd.RangeIndex(len(unique_service_id)),
 })

                clientId  mappedID
0     source user@domain         0
1              U898@DOM1         1
2               U12@DOM1         2
3               U14@DOM1         3
4               U51@DOM1         4
...                  ...       ...
8684         U3998@C2106      8684
8685        U1718@C17388      8685
8686             U9298@?      8686
8687          U1066@DOM1      8687
8688          U3648@DOM1      8688

[8689 rows x 2 columns]


In [7]:
print(len(unique_client_id), len(unique_service_id))

8689 8215


In [8]:
client_service

,client,service
0,source user@domain,destination computer
1,U898@DOM1,C2944
2,U12@DOM1,C368
3,U14@DOM1,C421
4,U51@DOM1,C1487
...,...,...
3590231,U194@DOM1,C10
3590232,U2@DOM1,C7
3590233,U68@DOM1,C7
3590234,U94@DOM1,C7


In [9]:
# Perform merge to obtain the edges from users and movies:
client_id = pd.merge(client_service['client'], unique_client_id, left_on='client', right_on='clientId', how='left')
client_id = torch.from_numpy(client_id['mappedID'].values)
service_id = pd.merge(client_service['service'], unique_service_id, left_on='service', right_on='serviceId', how='left')
service_id = torch.from_numpy(service_id['mappedID'].values)

# following PyG semantics:
edge_index_client_to_service = torch.stack([client_id, service_id], dim=0)

print(edge_index_client_to_service)


tensor([[   0,    1,    2,  ...,   52,    8, 1030],
        [   0,    1,    2,  ...,    6,    6,   22]])


In [10]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T


In [11]:
# pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

In [12]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
data["client"].node_id = torch.arange(len(unique_client_id))
data["service"].node_id = torch.arange(len(unique_service_id))

data["client", "auth_attempt", "service"].edge_index = edge_index_client_to_service

Transform = T.ToUndirected()
data = Transform(data)

In [13]:
transform = T.RandomLinkSplit(
    num_val=0.1,  # TODO
    num_test=0.1,  # TODO
    disjoint_train_ratio=0.3,  # TODO
    neg_sampling_ratio=2,  # TODO
    add_negative_train_samples=False,  # TODO
    edge_types=("client", "auth_attempt", "service"),
    rev_edge_types=("service", "rev_auth_attempt", "client"),
)

train_data, val_data, test_data = transform(data)

In [14]:
from torch_geometric.loader import LinkNeighborLoader

# Define seed edges:
edge_label_index = train_data["client", "auth_attempt", "service"].edge_label_index
edge_label = train_data["client", "auth_attempt", "service"].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[20,10],  # TODO
    neg_sampling_ratio=2,  # TODO
    edge_label_index=(("client", "auth_attempt", "service"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  client={ node_id=[842] },
  service={ node_id=[487] },
  (client, auth_attempt, service)={
    edge_index=[2, 1959],
    edge_label=[384],
    edge_label_index=[2, 384],
    input_id=[128]
  },
  (service, rev_auth_attempt, client)={ edge_index=[2, 6468] }
)


In [15]:
from torch_geometric.nn import SAGEConv, to_hetero

In [16]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()

        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # Define a 2-layer GNN computation graph.
        # Use a *single* `ReLU` non-linearity in-between.
        # TODO:
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x
        # raise NotImplementedError

# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]

        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        # self.movie_lin = torch.nn.Linear(20, hidden_channels)
        self.user_emb = torch.nn.Embedding(data["client"].num_nodes, hidden_channels)
        self.movie_emb = torch.nn.Embedding(data["service"].num_nodes, hidden_channels)

        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)

        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "client": self.user_emb(data["client"].node_id),
          "service": self.movie_emb(data["service"].node_id),
        } 

        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
        x_dict = self.gnn(x_dict, data.edge_index_dict)

        pred = self.classifier(
            x_dict["client"],
            x_dict["service"],
            data["client", "auth_attempt", "service"].edge_label_index,
        )

        return pred

        
model = Model(hidden_channels=64)

print(model)

Model(
  (user_emb): Embedding(8689, 64)
  (movie_emb): Embedding(8215, 64)
  (gnn): GraphModule(
    (conv1): ModuleDict(
      (client__auth_attempt__service): SAGEConv(64, 64, aggr=mean)
      (service__rev_auth_attempt__client): SAGEConv(64, 64, aggr=mean)
    )
    (conv2): ModuleDict(
      (client__auth_attempt__service): SAGEConv(64, 64, aggr=mean)
      (service__rev_auth_attempt__client): SAGEConv(64, 64, aggr=mean)
    )
  )
  (classifier): Classifier()
)


In [17]:
import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 8):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()

        
        # print(sampled_data)

        # TODO: Move `sampled_data` to the respective `device`
        sampled_data = sampled_data.to(device)
        # TODO: Run `forward` pass of the model
        pred = model.forward(sampled_data)

        # print(pred.size() == sampled_data["user", "rates", "movie"].edge_label.size())
        # print(sampled_data["user", "rates", "movie"].edge_label)
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`
        
        loss = F.binary_cross_entropy_with_logits(pred, sampled_data["client", "auth_attempt", "service"].edge_label)

        # raise NotImplementedError

        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


  1%|▎                                                       | 37/6732 [01:55<5:48:34,  3.12s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(model, '../lm-vol/torchmodelntlmpika')